# Building a book

## Load text

In [56]:
import spacy
nlp_ru = spacy.load("ru_core_news_lg")
def split_into_chunks(text: str, max_chars: int = 300) -> list:
    doc = nlp_ru(text)
    sentences = [sent.text.strip() for sent in doc.sents]

    chunks = []
    current_chunk = ""
    for sent in sentences:
        if len(current_chunk) + len(sent) <= max_chars:
            current_chunk += (" " if current_chunk else "") + sent
        else:
            chunks.append(current_chunk)
            current_chunk = sent
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [57]:
from pathlib import Path
def load_text_from_file(filepath: str) -> str:
    return Path(filepath).read_text(encoding='utf-8')
text_ru = load_text_from_file("./merge/text_ru.txt")
chunks_ru = split_into_chunks(text_ru.replace("\n", " "), max_chars=300)

In [58]:
len(chunks_ru)

2919

## Create dataframe

In [59]:
import pandas as pd
rus_df = pd.DataFrame({ "chunk": chunks_ru })

In [60]:
rus_df.shape

(2919, 1)

## Add column for translation

In [61]:
from googletrans import Translator

translator = Translator()
async def translate_russian_to_spanish(text: str) -> str:
    result = await translator.translate(text, src="ru", dest="es")
    return result.text

In [62]:
rus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   chunk   2919 non-null   object
dtypes: object(1)
memory usage: 22.9+ KB


In [63]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
from concurrent.futures import ThreadPoolExecutor

In [64]:
from tqdm.asyncio import tqdm_asyncio
# Wrapper to run async function in thread-friendly context
async def translate_all(chunks, max_workers=10):
    semaphore = asyncio.Semaphore(max_workers)
    async def run_chunk(text):
        async with semaphore:
            return await translate_russian_to_spanish(text)

    # Run in batches
    tasks = [run_chunk(text) for text in chunks]
    return await tqdm_asyncio.gather(*tasks)

In [65]:
# Run translation asynchronously and assign to new column
translations = asyncio.run(translate_all(rus_df["chunk"].tolist(), max_workers=20))
rus_df["translation"] = translations

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2919/2919 [01:27<00:00, 33.45it/s]


In [66]:
rus_df.tail(3)

,chunk,translation
2916,Макондо был уже почти весь перемолот в пыль и ...,Macondo ya estaba casi abrumado en el polvo y ...
2917,"Скользнул взглядом ниже, чтобы пропустить пред...",Miró a continuación para perder las prediccion...
2918,"Однако прежде чем взглянуть на последний стих,...","Sin embargo, antes de mirar el último versícul..."


In [67]:
rus_df.to_pickle("./merge/rus_df.pkl")

## Add embeddings

In [68]:
from dotenv import load_dotenv
load_dotenv()
from openai import AsyncOpenAI
client = AsyncOpenAI(max_retries=5)
from typing import List
async def get_embedding(text: str, model="text-embedding-3-small", **kwargs) -> List[float]:
    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")

    response = await client.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding

In [69]:
from tqdm.asyncio import tqdm_asyncio
# Wrapper to run async function in thread-friendly context
async def embed_all(chunks, max_workers=10):
    semaphore = asyncio.Semaphore(max_workers)
    async def run_chunk(text):
        async with semaphore:
            return await get_embedding(text)

    # Run in batches
    tasks = [run_chunk(text) for text in chunks]
    return await tqdm_asyncio.gather(*tasks)

In [70]:
rus_df.shape

(2919, 2)

In [71]:
subset = rus_df["translation"].tolist()
embeddings = asyncio.run(embed_all(subset, max_workers=20))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2919/2919 [00:52<00:00, 56.10it/s]


In [72]:
[len(embeddings), rus_df.shape]

[2919, (2919, 2)]

In [73]:
rus_df["embedding"] = embeddings

In [74]:
rus_df.to_pickle("./merge/rus_df.pkl")

In [75]:
rus_df.head(1)

,chunk,translation,embedding
0,"— Я хочу быть только с тобой, — заявил он. — С...","""Solo quiero estar contigo"", dijo. - Pronto le...","[0.02256019599735737, -0.029076769948005676, -..."


## Get target edition chunks

In [76]:
from more_itertools import split_at
def break_long_sentences(doc):
    sublists = list(" ".join(line) for line in split_at([d.text for d in doc], lambda x: x == ","))
    return [chunk + ',' if i < len(sublists) - 1 else chunk for i, chunk in enumerate(sublists)]

In [79]:
from itertools import chain
import es_core_news_sm
nlp_es = es_core_news_sm.load()
def get_es_chunks(text: str) -> list:
    doc = nlp_es(text)

    chunks = []
    current_chunk = ""
    parts = [break_long_sentences(s) for s in doc.sents]
    return list(chain.from_iterable(parts))

In [80]:
text_es = load_text_from_file("./merge/text_es.txt")
chunks_es = get_es_chunks(text_es.replace("\n", " "))

In [81]:
len(chunks_es)

13442

In [82]:
df_es = pd.DataFrame({ "chunk": chunks_es })

In [83]:
df_es.shape

(13442, 1)

In [ ]:
df_es.to_pickle("./merge/es_df.pkl")

In [120]:
rus_df['match'] = None
rus_df.loc[0]

chunk          — Я хочу быть только с тобой, — заявил он. — С...
translation    "Solo quiero estar contigo", dijo. - Pronto le...
embedding      [0.02256019599735737, -0.029076769948005676, -...
match                                                       None
Name: 0, dtype: object

In [109]:
rus_df['embedding'][:3]

0    [0.02256019599735737, -0.029076769948005676, -...
1    [0.0629948303103447, -0.008339416235685349, -0...
2    [0.05613021180033684, -0.00419988390058279, 0....
Name: embedding, dtype: object

In [106]:
import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [110]:
async def get_matching_fragment(embed, pointer):
    i=pointer+1
    last_score = -1
    while 1:
        score = cosine_similarity(
            embed,
            await get_embedding(" ".join(chunks_es[pointer:i]))
        )
        if (score < last_score):
            break
        last_score = score
        i += 1
    return (" ".join(chunks_es[pointer:i-1]), i-1, last_score)

In [127]:
pointer = 0
for (i, ru_embed) in enumerate(rus_df['embedding']):
    (es_chunk, new_pointer, score) = await get_matching_fragment(ru_embed, pointer)
    retries = 0
    while (score < 0.6) and (retries < 5):
        print(f"🔁 Retracing for fragment {i} | pointer {pointer} |️ {score}")
        pointer -= 1
        (es_chunk, new_pointer, score) = await get_matching_fragment(ru_embed, pointer)
        retries += 1
    pointer += retries
    retries = 0
    while (score < 0.6) and (retries < 5):
        print(f"🔁 Retracing for fragment {i} | pointer {pointer} |️ {score}")
        pointer += 1
        (es_chunk, new_pointer, score) = await get_matching_fragment(ru_embed, pointer)
        retries += 1
    if (score < 0.6):
        rus_df.to_pickle("./merge/rus_df_matched.pkl")
        break
    pointer = new_pointer
    rus_df.loc[i, "match"] = es_chunk
    if i % 10 == 0:
        rus_df.to_pickle("./merge/rus_df_matched.pkl")
        print(f"✅ Data stored to file")
    print(f"✅ fragment {i} |️ {score}")

✅ Data stored to file
✅ fragment 0 |️ 0.7487427146261971
✅ fragment 1 |️ 0.8651565082225524
✅ fragment 2 |️ 0.6724228675231383
✅ fragment 3 |️ 0.648529889039089
✅ fragment 4 |️ 0.6051339958340011
✅ fragment 5 |️ 0.7873766153073934
✅ fragment 6 |️ 0.7354530715335702
✅ fragment 7 |️ 0.7372134644935845
✅ fragment 8 |️ 0.8817601594716075
✅ fragment 9 |️ 0.8325358072585453
✅ Data stored to file
✅ fragment 10 |️ 0.8162378863515477
🔁 Retracing for fragment 11 | pointer 51 |️ 0.4365744334970984
🔁 Retracing for fragment 11 | pointer 50 |️ 0.4596365200876108
✅ fragment 11 |️ 0.7607178223732834
✅ fragment 12 |️ 0.7160444095225752
✅ fragment 13 |️ 0.6442303363845537
✅ fragment 14 |️ 0.6377902251924785
✅ fragment 15 |️ 0.7045976805923272
✅ fragment 16 |️ 0.7644475524734028
✅ fragment 17 |️ 0.809622991553264
✅ fragment 18 |️ 0.7864550272925133
✅ fragment 19 |️ 0.8013021441533662
🔁 Retracing for fragment 20 | pointer 95 |️ 0.3896197073916885
🔁 Retracing for fragment 20 | pointer 94 |️ 0.4064374467262

In [117]:
rus_df = pd.read_pickle("./merge/rus_df.pkl")

In [122]:
rus_df.shape

(2919, 4)

In [129]:
[rus_df.loc[127, "chunk"], rus_df.loc[127, "match"]]

['Они болтали без умолку, перебивая друг друга, рассказывали до обалдения одни и те же старые анекдоты, участвовали в шутейном действе про белого бычка, а если все молчали, рассказчик говорил, что он просил не молчать, а сказать, хотят ли они слушать сказку про белого бычка, и никто не мог уйти, потому что рассказчик говорил, что он не просил уходить, а лишь ответить, хотят ли они слушать сказку про белого бычка, и так без конца, все ночи напролет, загнав себя в порочный круг пустопорожних фраз.',
 'Se reunían a conversar sin tregua, a repetirse durante horas y horas los mismos chistes, a complicar hasta los límites de la exasperación el cuento del gallo capón, que era un juego infinito en que el narrador preguntaba si querían que les contara el cuento del gallo capón, y cuando contestaban que sí, el narrador decía que no les había pedido que dijeran que sí,']

In [132]:
rus_df.loc[128, "chunk"]

'Когда Хосе Аркадио Буэндия понял, что поветрие распространилось по всему городку, он собрал отцов семейств и рассказал им все, что знал о бессонной болезни, и было решено принять меры, чтобы помешать заразе перекинуться на соседние селения.'

In [155]:
[rus_df.loc[128, "chunk"], chunks_es[563]]

['Когда Хосе Аркадио Буэндия понял, что поветрие распространилось по всему городку, он собрал отцов семейств и рассказал им все, что знал о бессонной болезни, и было решено принять меры, чтобы помешать заразе перекинуться на соседние селения.',
 'Cuando José Arcadio Buendía se dio cuenta de que la peste había invadido el pueblo,']

In [ ]:
pointer

In [154]:
rus_df['embedding'][128].index

<function list.index(value, start=0, stop=9223372036854775807, /)>

In [158]:
pointer = 563
for (i, ru_embed) in enumerate(rus_df['embedding'][128:], start=128):
    (es_chunk, new_pointer, score) = await get_matching_fragment(ru_embed, pointer)
    retries = 0
    while (score < 0.6) and (retries < 5):
        print(f"🔁 Retracing for fragment {i} | pointer {pointer} |️ {score}")
        pointer -= 1
        (es_chunk, new_pointer, score) = await get_matching_fragment(ru_embed, pointer)
        retries += 1
    pointer += retries
    retries = 0
    while (score < 0.6) and (retries < 5):
        print(f"🔁 Retracing for fragment {i} | pointer {pointer} |️ {score}")
        pointer += 1
        (es_chunk, new_pointer, score) = await get_matching_fragment(ru_embed, pointer)
        retries += 1
    if (score < 0.6):
        rus_df.to_pickle("./merge/rus_df_matched.pkl")
        break
    pointer = new_pointer
    rus_df.loc[i, "match"] = es_chunk
    if i % 10 == 0:
        rus_df.to_pickle("./merge/rus_df_matched.pkl")
        print(f"✅ Data stored to file")
    print(f"✅ fragment {i} |️ {score}")

✅ fragment 128 |️ 0.6930626802641963
✅ fragment 129 |️ 0.7282699385569497
🔁 Retracing for fragment 130 | pointer 569 |️ 0.44161795026677586
✅ Data stored to file
✅ fragment 130 |️ 0.7641808259784947
✅ fragment 131 |️ 0.7393764084549267
✅ fragment 132 |️ 0.8678477350338707
✅ fragment 133 |️ 0.6948443729361155
✅ fragment 134 |️ 0.7533633242103599
✅ fragment 135 |️ 0.7345502172055403
🔁 Retracing for fragment 136 | pointer 589 |️ 0.5540566195515938
🔁 Retracing for fragment 136 | pointer 588 |️ 0.497206988233643
🔁 Retracing for fragment 136 | pointer 587 |️ 0.4612396626505195
🔁 Retracing for fragment 136 | pointer 586 |️ 0.43541941298402626
🔁 Retracing for fragment 136 | pointer 585 |️ 0.366250138413826
🔁 Retracing for fragment 136 | pointer 589 |️ 0.3857825252446326
🔁 Retracing for fragment 136 | pointer 590 |️ 0.53834614672186
🔁 Retracing for fragment 136 | pointer 591 |️ 0.3075580660798289
🔁 Retracing for fragment 136 | pointer 592 |️ 0.29326566709477087
🔁 Retracing for fragment 136 | po

In [163]:
rus_df.loc[136, "chunk"]

'Тогда он прилепил на них нужные наклейки, и стоило только взглянугь на ярлык, как сразу делалось понятно, что это за штука.'

In [165]:
chunks_es[589:591]

['Entonces las marcó con el nombre respectivo,',
 'de modo que le bastaba con leer la inscripción para identificarlas .']

In [170]:
cosine_similarity(rus_df.loc[136, "embedding"], await get_embedding(" ".join(chunks_es[589:591])))

np.float64(0.5540566195515938)

In [188]:
async def start_from(pointer, ru_chunk_index):
    for (i, ru_embed) in enumerate(rus_df['embedding'][ru_chunk_index:], start=ru_chunk_index):
        (es_chunk, new_pointer, score) = await get_matching_fragment(ru_embed, pointer)
        retries = 0
        while (score < 0.6) and (retries < 5):
            print(f"🔁 Retracing for fragment {i} | pointer {pointer} |️ {score}")
            pointer -= 1
            (es_chunk, new_pointer, score) = await get_matching_fragment(ru_embed, pointer)
            retries += 1
        pointer += retries
        retries = 0
        while (score < 0.6) and (retries < 5):
            print(f"🔁 Retracing for fragment {i} | pointer {pointer} |️ {score}")
            pointer += 1
            (es_chunk, new_pointer, score) = await get_matching_fragment(ru_embed, pointer)
            retries += 1
        if (score < 0.6):
            rus_df.to_pickle("./merge/rus_df_matched.pkl")
            return pointer
        pointer = new_pointer
        rus_df.loc[i, "match"] = es_chunk
        if i % 10 == 0:
            rus_df.to_pickle("./merge/rus_df_matched.pkl")
            print(f"✅ Data stored to file")
        print(f"✅ fragment {i} |️ {score}")

In [179]:
rus_df.loc[136, "match"] = " ".join(chunks_es[589:591])

In [180]:
rus_df.loc[136, "match"]

'Entonces las marcó con el nombre respectivo, de modo que le bastaba con leer la inscripción para identificarlas .'

In [182]:
[rus_df.loc[137, "chunk"], chunks_es[591]]

['Когда отец с тревогой сказал ему, что забыл почти все, даже самые сильные впечатления детства, Аурелиано сообщил ему о своем методе, и Хосе Аркадио Буэндия стал навешивать ярлыки на все домашние вещи, а потом ввел эту практику и во всем городке.',
 'Cuando su padre le comunicó su alarma por haber olvidado hasta los hechos más impresionantes de su niñez,']

In [184]:
pointer = 591

In [191]:
new_pointer = await start_from(591, 137)

✅ fragment 137 |️ 0.8301769096299978
✅ fragment 138 |️ 0.6725626270295503
🔁 Retracing for fragment 139 | pointer 603 |️ 0.1667428216207925
🔁 Retracing for fragment 139 | pointer 602 |️ 0.15794865924824872
🔁 Retracing for fragment 139 | pointer 601 |️ 0.25359684721021236
🔁 Retracing for fragment 139 | pointer 600 |️ 0.22418672706573123
🔁 Retracing for fragment 139 | pointer 599 |️ 0.2555706660125172
🔁 Retracing for fragment 139 | pointer 603 |️ 0.2675247307646622
🔁 Retracing for fragment 139 | pointer 604 |️ 0.12015919138697674
🔁 Retracing for fragment 139 | pointer 605 |️ 0.05846072897701642
🔁 Retracing for fragment 139 | pointer 606 |️ 0.13907983781408062
✅ fragment 139 |️ 0.6316412498424689
🔁 Retracing for fragment 140 | pointer 611 |️ 0.29198690829683827
✅ Data stored to file
✅ fragment 140 |️ 0.7959809243475134
✅ fragment 141 |️ 0.7591700049891225
✅ fragment 142 |️ 0.8047103791646074
✅ fragment 143 |️ 0.8100803129783722
✅ fragment 144 |️ 0.6480599415390196
✅ fragment 145 |️ 0.72881

In [215]:
rus_df.loc[156, "chunk"]

'Ослепительная вспышка радости увлажнила ему глаза слезами раньше, чем он увидел себя в идиотской комнате, где все вещи имели ярлычки с названиями; раньше, чем он устыдился всех благоглупостей, начертанных на стенах, и даже раньше, чем узнал пришедшего. А пришел Мелькиадес.'

In [216]:
[c for c in enumerate(chunks_es[654:684])]

[(0, 'no con el olvido remediable del corazón,'),
 (1, 'sino con otro olvido más cruel e irrevocable que él conocía muy bien,'),
 (2, 'porque era el olvido de la muerte .'),
 (3, 'Entonces comprendió .'),
 (4, 'Abrió la maleta atiborrada de objetos indescifrables,'),
 (5, 'y de entre ellos sacó un maletín con muchos frascos .'),
 (6, 'Le dio a beber a José Arcadio Buendía una sustancia de color apacible,'),
 (7, 'y la luz se hizo en su memoria .'),
 (8, 'Los ojos se le humedecieron de llanto,'),
 (9,
  'antes de verse a sí rnismo en una sala absurda donde los objetos estaban marcados,'),
 (10,
  'y antes de avergonzarse de las solemnes tonterías escritas en las paredes,'),
 (11,
  'y aun antes de reconocer al recién llegado en un deslumbrante resplandor de alegría .'),
 (12, 'Era Melquíades .'),
 (13, 'Mientras Macondo celebraba la reconquista de los recuerdos,'),
 (14,
  'José Arcadio Buendía y Melquíades le sacudieron el polvo a su vieja amistad .'),
 (15, 'EI gitano iba dispuesto a 

In [218]:
[rus_df.loc[156, "chunk"], chunks_es[662]]

['Ослепительная вспышка радости увлажнила ему глаза слезами раньше, чем он увидел себя в идиотской комнате, где все вещи имели ярлычки с названиями; раньше, чем он устыдился всех благоглупостей, начертанных на стенах, и даже раньше, чем узнал пришедшего. А пришел Мелькиадес.',
 'Los ojos se le humedecieron de llanto,']

In [219]:
pointer = await start_from(662, 156)

✅ fragment 156 |️ 0.84126885381759
✅ fragment 157 |️ 0.8739175452208883
✅ fragment 158 |️ 0.7138172942856245
✅ fragment 159 |️ 0.7460890560489518
✅ Data stored to file
✅ fragment 160 |️ 0.7994870370463891
✅ fragment 161 |️ 0.7868204318395056
✅ fragment 162 |️ 0.6363919186608169
✅ fragment 163 |️ 0.8504959677150539
✅ fragment 164 |️ 0.7710934526607249
✅ fragment 165 |️ 0.8188025506409532
✅ fragment 166 |️ 0.7091167693915742
✅ fragment 167 |️ 0.7137345840336297
✅ fragment 168 |️ 0.8363806562396285
✅ fragment 169 |️ 0.877046839664372
✅ Data stored to file
✅ fragment 170 |️ 0.7699487747638002
✅ fragment 171 |️ 0.7350295990446368
✅ fragment 172 |️ 0.8413517509722384
✅ fragment 173 |️ 0.8186312329999182
✅ fragment 174 |️ 0.7657661346913919
✅ fragment 175 |️ 0.8024177944183715
✅ fragment 176 |️ 0.7563025870696723
🔁 Retracing for fragment 177 | pointer 736 |️ 0.5346732338505763
✅ fragment 177 |️ 0.605843706024766
✅ fragment 178 |️ 0.8206607889941836
✅ fragment 179 |️ 0.8013018732072053
✅ Data 

In [220]:
rus_df.loc[218, "chunk"]

'— В этом городе мы обходимся без бумаг, — сказал он, не теряя присутствия духа. — Зарубите себе на носу: нам не нужен никакой управитель, мы сами прекрасно здесь управляемся.'

In [222]:
" ".join(chunks_es[934:937])

'— En este pueblo no mandamos con papeles — dijo sin perder la calma — . Y para que 10 sepa de una vez, no necesitamos ningún corregidor porque aquí no hay nada que corregir .'

In [223]:
cosine_similarity(rus_df.loc[218, "embedding"], await get_embedding(" ".join(chunks_es[934:937])))

np.float64(0.5767228447939377)

In [232]:
rus_df.loc[218, "match"] = " ".join(chunks_es[934:937])

In [249]:
rus_df.loc[218, "chunk"] = backup_df.loc[218, "chunk"]

In [229]:
chunks_es[937:943]

['Ante la impavidez de don Apolinar Moscote,',
 'siempre sin levantar la voz,',
 'hizo un pormenorizado recuento de cómo habían fundado la aldea,',
 'de cómo se habían repartido la tierra,',
 'abierto los caminos e introducido las mejoras que les había ido exigiendo la necesidad,',
 'sin haber molestado a gobierno alguno y sin que nadie los molestara .']

In [237]:
rus_df.loc[219, "match"] = " ".join(chunks_es[937:943])

In [251]:
rus_df.loc[219]

chunk          И Хосе Аркадио Буэндия, глядя на невозмутимого...
translation    Y José Arkadio Buendy, mirando el Muscovite Do...
embedding      [0.054191216826438904, 0.005699383094906807, 0...
match          Ante la impavidez de don Apolinar Moscote, sie...
Name: 219, dtype: object

In [241]:
backup_df = pd.read_pickle("./merge/rus_df.pkl")

In [252]:
rus_df.loc[220, "chunk"]

'«Мы живем столь тихо и мирно, что даже смерть нас стороной обходит, — сказал он. — вы сами видите, что тут нет кладбища». На правительство он обиды не держит за то, что им не помогли.'

In [254]:
chunks_es[943:953]

['« Somos tan pacíficos que ni siquiera nos hemos muerto de muerte natural »,',
 'dijo .',
 '« Ya ve que todavía no tenernos cementerio .',
 '» No se dolió de que el gobierno no los hubiera ayudado .',
 'Al contrario,',
 'se alegraba de que hasta entonces los hubiera dejado crecer en paz,',
 'y esperaba que así los siguiera dejando,',
 'porque ellos no habían fundado un pueblo para que el primer advenedizo les fuera a decir 10 que debían hacer .',
 'Don Apolinar Moscote se había puesto un saco de dril,',
 'blanco como sus pantalones,']

In [255]:
pointer = await start_from(943, 220)

✅ Data stored to file
✅ fragment 220 |️ 0.8316591321946442
🔁 Retracing for fragment 221 | pointer 947 |️ 0.43276087748417413
🔁 Retracing for fragment 221 | pointer 946 |️ 0.4526499407240603
🔁 Retracing for fragment 221 | pointer 945 |️ 0.4492925332578794
🔁 Retracing for fragment 221 | pointer 944 |️ 0.462957299060328
🔁 Retracing for fragment 221 | pointer 943 |️ 0.4380571279731144
🔁 Retracing for fragment 221 | pointer 947 |️ 0.3832808232319134
🔁 Retracing for fragment 221 | pointer 948 |️ 0.4198677839811001
🔁 Retracing for fragment 221 | pointer 949 |️ 0.45429494401236886
🔁 Retracing for fragment 221 | pointer 950 |️ 0.36757211517546723
🔁 Retracing for fragment 221 | pointer 951 |️ 0.22127622875664194


In [256]:
rus_df.loc[221, "chunk"]

'Напротив, очень доволен тем, что до сих пор никто не вставлял палки в колеса и что впредь так оно и будет, потому как они основали город не для того, чтобы первый пришлый указывал им, за какие браться дела.'

In [259]:
" ".join(chunks_es[947:951])

'Al contrario, se alegraba de que hasta entonces los hubiera dejado crecer en paz, y esperaba que así los siguiera dejando, porque ellos no habían fundado un pueblo para que el primer advenedizo les fuera a decir 10 que debían hacer .'

In [260]:
cosine_similarity(rus_df.loc[221, "embedding"], await get_embedding(" ".join(chunks_es[947:951])))

np.float64(0.46224171513347845)

In [261]:
[rus_df.loc[221, "translation"], " ".join(chunks_es[947:951])]

['Por el contrario, estoy muy contento de que hasta ahora nadie haya insertado palos en las ruedas y que continúe así, porque encontraron la ciudad no para que el primer periódico les mostrara qué tipo de asuntos asumir.',
 'Al contrario, se alegraba de que hasta entonces los hubiera dejado crecer en paz, y esperaba que así los siguiera dejando, porque ellos no habían fundado un pueblo para que el primer advenedizo les fuera a decir 10 que debían hacer .']

In [262]:
rus_df.loc[221, "match"] = " ".join(chunks_es[947:951])

In [266]:
rus_df.loc[222, "chunk"]

'Дон Аполинар Москоте аккуратно влез в рукава полотняного кителя, белого, как и брюки, и тщательно застегнулся. — Так что, если вы желаете здесь остаться и жить, как живут обычные нормальные люди, милости просим, — закончил Хосе Аркадио Буэндия.'

In [268]:
pointer = await start_from(951, 222)

✅ fragment 222 |️ 0.7374092794010507
✅ fragment 223 |️ 0.8353431279319482
✅ fragment 224 |️ 0.8376013208406766
✅ fragment 225 |️ 0.6612123398270768
✅ fragment 226 |️ 0.7399253944601267
✅ fragment 227 |️ 0.6126009909791045
✅ fragment 228 |️ 0.6784404700813653
✅ fragment 229 |️ 0.7894329062859442
✅ Data stored to file
✅ fragment 230 |️ 0.8983466511375552
✅ fragment 231 |️ 0.7170314292878094
✅ fragment 232 |️ 0.8156426599580765
🔁 Retracing for fragment 233 | pointer 1009 |️ 0.39803090835436683
🔁 Retracing for fragment 233 | pointer 1008 |️ 0.44738740810456845
🔁 Retracing for fragment 233 | pointer 1007 |️ 0.45715926962550885
🔁 Retracing for fragment 233 | pointer 1006 |️ 0.42388780386512026
🔁 Retracing for fragment 233 | pointer 1005 |️ 0.4683768214450446
🔁 Retracing for fragment 233 | pointer 1009 |️ 0.4820842897003066
✅ fragment 233 |️ 0.773265037114969
✅ fragment 234 |️ 0.8160662713995487
✅ fragment 235 |️ 0.6952131839288181
✅ fragment 236 |️ 0.7356238391137593
✅ fragment 237 |️ 0.7140

In [271]:
rus_df.shape

(2919, 4)